In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

%pwd

'/Users/dongzheng/Codes/PythonWorkspace/CSE5005-Project/catboost_model'

In [2]:
# num_users=5000
# eye=np.eye(num_users)
# def onehot_encode(x):
#     return eye[x]

In [3]:
def gen_xy(sequence):
    """
    Returns
    ---
    x: (num_samples, 5)
    y: (num_samples,) 1-d vec for labels
    """

    x, y = [], []
    for i in range(len(sequence) - 2):
        x.append(sequence[i : i + 2])
        y.append(sequence[i + 1])

    x = np.array(x)
    x = x.reshape(x.shape[0], x.shape[1] * x.shape[2])
    x = x[:, :5]
    y = np.array(y)

    if len(y.shape) > 1:
        y = y[:, -1]

    return x, y

sequence = pd.read_pickle("../data/data_100000_distr.pkl")[
    ["from_user_id", "to_user_id", "label"]
].values

x, y=gen_xy(sequence)

print(x.shape)
print(y.shape)

(99998, 5)
(99998,)


In [4]:
train_size=0.7
val_size=0.1

split1 = int(len(x) * train_size)
split2 = int(len(sequence) * (train_size + val_size))

x_train, y_train = x[:split1], y[:split1]
x_val, y_val = x[split1:split2], y[split1:split2]
x_test, y_test = x[split2:], y[split2:]

print(f"Trainset:\tx-{x_train.shape}\ty-{y_train.shape}")
print(f"Valset:  \tx-{x_val.shape}  \ty-{y_val.shape}")
print(f"Testset:\tx-{x_test.shape}\ty-{y_test.shape}")

Trainset:	x-(69998, 5)	y-(69998,)
Valset:  	x-(10002, 5)  	y-(10002,)
Testset:	x-(19998, 5)	y-(19998,)


In [5]:
from sklearn.svm import SVC

svc=SVC()
svc.fit(x_train, y_train)

y_train_pred=svc.predict(x_train)
y_val_pred=svc.predict(x_val)
y_pred=svc.predict(x_test)

print(accuracy_score(y_train, y_train_pred))
print(accuracy_score(y_val, y_val_pred))
print(accuracy_score(y_test, y_pred))

0.5125289293979828
0.5043991201759648
0.4963496349634963


In [6]:
from sklearn.neighbors import KNeighborsClassifier

train_acc=[]
test_acc=[]
for k in range(1, 10):
    knn=KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)

    train_acc.append(knn.score(x_train, y_train))
    test_acc.append(knn.score(x_test, y_test))
    
test_acc

[0.5034003400340034,
 0.49964996499649966,
 0.4977997799779978,
 0.4967496749674967,
 0.5,
 0.4968996899689969,
 0.4973497349734973,
 0.4952995299529953,
 0.4963996399639964]

In [9]:
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)

y_train_pred=knn.predict(x_train)
y_val_pred=knn.predict(x_val)
y_pred=knn.predict(x_test)

print(accuracy_score(y_train, y_train_pred))
print(accuracy_score(y_val, y_val_pred))
print(accuracy_score(y_test, y_pred))

0.6882053772964942
0.49750049990002
0.5


In [28]:
from sklearn import tree

clf=tree.DecisionTreeClassifier(criterion="gini", max_depth=3)
clf=clf.fit(x_train, y_train)

y_train_pred=clf.predict(x_train)
y_val_pred=clf.predict(x_val)
y_pred=clf.predict(x_test)

print(accuracy_score(y_train, y_train_pred))
print(accuracy_score(y_val, y_val_pred))
print(accuracy_score(y_test, y_pred))

0.6675762164633275
0.6666666666666666
0.6712671267126713


In [39]:
et=tree.ExtraTreeClassifier(criterion="gini", splitter="best", max_depth=5)
et=et.fit(x_train, y_train)

y_train_pred=et.predict(x_train)
y_val_pred=et.predict(x_val)
y_pred=et.predict(x_test)

print(accuracy_score(y_train, y_train_pred))
print(accuracy_score(y_val, y_val_pred))
print(accuracy_score(y_test, y_pred))

0.6676905054430127
0.6662667466506699
0.6712171217121712


In [58]:
from sklearn.ensemble import RandomForestClassifier

rf=RandomForestClassifier(n_estimators=50, max_depth=5)
rf=rf.fit(x_train, y_train)

y_train_pred=rf.predict(x_train)
y_val_pred=rf.predict(x_val)
y_pred=rf.predict(x_test)

print(accuracy_score(y_train, y_train_pred))
print(accuracy_score(y_val, y_val_pred))
print(accuracy_score(y_test, y_pred))

0.6674619274836424
0.6666666666666666
0.6712671267126713


In [1]:
%run sklearn_models.py

Trainset:	x-(69838, 5)	y-(69838,)
Valset:  	x-(9978, 5)  	y-(9978,)
Testset:	x-(19953, 5)	y-(19953,)
<class 'sklearn.neighbors._classification.KNeighborsClassifier'>
0.8129098771442481
0.7568651032270997
0.7425950984814313
<class 'sklearn.tree._classes.DecisionTreeClassifier'>
0.7963429651479137
0.7908398476648627
0.7761740089209642
<class 'sklearn.tree._classes.ExtraTreeClassifier'>
0.7968154872705404
0.7909400681499299
0.7762742444745151
<class 'sklearn.ensemble._forest.RandomForestClassifier'>
0.7963429651479137
0.7908398476648627
0.7761740089209642
<class 'sklearn.svm._classes.SVC'>
0.7963429651479137
0.7908398476648627
0.7761740089209642
